자연어 처리 순서
```
1. 데이터 수집
2. 데이터 전처리
  - 토큰화 : 텍스트를 단어,또는 문장으로 나누는과정
  - 불용어 제거 : 분석에 불필요한 단어제거
  - 어간 추출 및 표제어 추출 : 단어의 원형을 찾기
  - 정규화 : 텍스트를 일관된 형식으로 변환
3. 피쳐 엔지니어링
  텍스트  데이터를 학습가능한 적합한 형태로 변환(수치화)
  - BoW : 단어의 빈도를 기반으로 벡터를 생성
  - TF-IDF : 단어의 중요도를 반영(희소벡터)
  - 워드 임베딩 : 문맥을 고려해서 고차원 벡터로(밀집벡터)
4. 모델 선택 학습
  - 머신러닝
  - 인공신경망
  - RMM LSTM GRU 등등
5. 모델 평가
```

2. 데이터 전처리
  - 토큰화

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize, sent_tokenize
text = '자연어 처리는 재미있습니다. 다양한 분야에서 활용할 수 있습니다.'
# 단어 토큰화
word_token = word_tokenize(text)
print(f'단어 토큰 : {word_token}')
# 문장 토큰
sent_token = sent_tokenize(text)
print(f'문장 토큰 : {sent_token}')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


단어 토큰 : ['자연어', '처리는', '재미있습니다', '.', '다양한', '분야에서', '활용할', '수', '있습니다', '.']
문장 토큰 : ['자연어 처리는 재미있습니다.', '다양한 분야에서 활용할 수 있습니다.']


In [ ]:
import pandas as pd
url = 'https://gist.githubusercontent.com/spikeekips/40eea22ef4a89f629abd87eed535ac6a/raw/4f7a635040442a995568270ac8156448f2d1f0cb/stopwords-ko.txt'
ko_stopwords = pd.read_csv(url, header=None).values
ko_stopwords = [w for word in ko_stopwords for w in word]
ko_stopwords[:2]
filtered_tokens = [word for word in word_token if word not in ko_stopwords and len(word)>=2]
filtered_tokens

['자연어', '처리는', '재미있습니다', '다양한', '분야에서', '활용할', '있습니다']

2. 데이터 전처리  
  - 어간 추출 및 표제어 추출 : 단어의 원형을 찾기

In [ ]:
! pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 22.2 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
# 형태소 분석기
okt = Okt()
# 어간 추출
stemed_tokens = [okt.morphs(word) for word in filtered_tokens]
stemed_tokens = [ word for word_lists in stemed_tokens for word in word_lists ]
print(f'어간 추출 : {stemed_tokens}')
# 표제어 추출
pos_tokens = [okt.pos(word) for word in filtered_tokens]
print(f'표제어 추출 : {pos_tokens}')

어간 추출 : ['자연어', '처리', '는', '재미있습니다', '다양한', '분야', '에서', '활용', '할', '있습니다']
표제어 추출 : [[('자연어', 'Noun')], [('처리', 'Noun'), ('는', 'Josa')], [('재미있습니다', 'Adjective')], [('다양한', 'Adjective')], [('분야', 'Noun'), ('에서', 'Josa')], [('활용', 'Noun'), ('할', 'Verb')], [('있습니다', 'Adjective')]]


In [ ]:
# 영어인경우
import re
def normalize_text(text):
  # 소문자 변환
  text = text.lower()
  # 숫자 제거 2024년 12월 25살 홍길동 .....
  # 숫자가 있었다는 사실만 알리기 위해서 0년 0월 0살 홍길동 .....
  re.sub = re.sub(r'\d+', '0', text) # Wd+ 숫자가 연속적인 패턴
  # 특수문자 제거
  text = re.sub(f'[^\w\s]','',text) # \w --> [a-zA-Z0-9], \s --> 스페이스,탭 줄바꿈 제거
  return text

3. 피쳐 엔지니어링
- 텍스트  데이터를 학습가능한 적합한 형태로 변환(수치화)
  - BoW : 단어의 빈도를 기반으로 벡터를 생성  

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
text = ['자연어 처리는 재미있습니다. 다양한 분야에서 활용할 수 있습니다.', '다양한 분야에서 활용할 수 있습니다.']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
print(f'BOW 벡터 : {X.toarray()}')
print(f'특성이름 : {vectorizer.get_feature_names_out()}')

BOW 벡터 : [[1 1 1 1 1 1 1]
 [1 1 1 0 0 0 1]]
특성이름 : ['다양한' '분야에서' '있습니다' '자연어' '재미있습니다' '처리는' '활용할']


3. 피쳐 엔지니어링
  텍스트  데이터를 학습가능한 적합한 형태로 변환(수치화)
  - TF-IDF : 단어의 중요도를 반영(희소벡터)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfk = TfidfVectorizer()
X = tfk.fit_transform(text)
print(f'TF-IDF 벡터 : {X.toarray()}')
print(f'특성이름 : {vectorizer.get_feature_names_out()}')

TF-IDF 벡터 : [[0.3174044  0.3174044  0.3174044  0.44610081 0.44610081 0.44610081
  0.3174044 ]
 [0.5        0.5        0.5        0.         0.         0.
  0.5       ]]
특성이름 : ['다양한' '분야에서' '있습니다' '자연어' '재미있습니다' '처리는' '활용할']


3. 피쳐 엔지니어링
  텍스트  데이터를 학습가능한 적합한 형태로 변환(수치화)
  - BoW : 단어의 빈도를 기반으로 벡터를 생성
  - TF-IDF : 단어의 중요도를 반영(희소벡터)
  - 워드 임베딩 : 문맥을 고려해서 고차원 벡터로(밀집벡터)

In [ ]:
# requires numpy<3.0.0,>=2.0.0,
!pip uninstall numpy
!pip install numpy==1.23.5

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.11
    /usr/local/lib/python3.11/dist-packages/numpy-1.23.5.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-742d56dc.3.20.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/lo

In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is 

In [ ]:
from gensim.models import Word2Vec
# [ ['자연어','처리','재미있습니다'],[]   ]
text = [ ['자연어','처리','재미있습니다'],['자연어','처리','응용','분야']   ]
model = Word2Vec(text,vector_size=100,window=5,min_count=1)
# 단어벡터
word_vector = model.wv
print(f'key_to_index : {word_vector }')
print(f'key_to_index : {word_vector.key_to_index }')
print(f"자연어 벡터 : {word_vector['자연어']}, {len(word_vector['자연어'])}")

key_to_index : KeyedVectors<vector_size=100, 5 keys>
key_to_index : {'처리': 0, '자연어': 1, '분야': 2, '응용': 3, '재미있습니다': 4}
자연어 벡터 : [-8.6196875e-03  3.6657380e-03  5.1898835e-03  5.7419385e-03
  7.4669183e-03 -6.1676754e-03  1.1056137e-03  6.0472824e-03
 -2.8400505e-03 -6.1735227e-03 -4.1022300e-04 -8.3689485e-03
 -5.6000124e-03  7.1045388e-03  3.3525396e-03  7.2256695e-03
  6.8002474e-03  7.5307419e-03 -3.7891543e-03 -5.6180597e-04
  2.3483764e-03 -4.5190323e-03  8.3887316e-03 -9.8581640e-03
  6.7646410e-03  2.9144168e-03 -4.9328315e-03  4.3981876e-03
 -1.7395747e-03  6.7113843e-03  9.9648498e-03 -4.3624435e-03
 -5.9933780e-04 -5.6956373e-03  3.8508223e-03  2.7866268e-03
  6.8910765e-03  6.1010956e-03  9.5384968e-03  9.2734173e-03
  7.8980681e-03 -6.9895042e-03 -9.1558648e-03 -3.5575271e-04
 -3.0998408e-03  7.8943167e-03  5.9385742e-03 -1.5456629e-03
  1.5109634e-03  1.7900408e-03  7.8175711e-03 -9.5101865e-03
 -2.0553112e-04  3.4691966e-03 -9.3897223e-04  8.3817719e-03
  9.0107834e-03  6

4. 모델 선택 학습
  - 머신러닝
  - 인공신경망
  - RMM LSTM GRU 등등

In [ ]:
# 1. 데이터 로드
import pandas as pd

splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'test': 'plain_text/test-00000-of-00001.parquet', 'unsupervised': 'plain_text/unsupervised-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["train"])
df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [ ]:
# 2. 데이터 전처리
# 2-1 토큰화
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize, sent_tokenize
df['token'] = df['text'].apply(word_tokenize)
# 2-2 불용어제거
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['filterred_tokens'] = df['token'].apply(lambda x: [word for word in x if word not in stop_words])
# 2-3 어간 추출 및 표제어 추출
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df['stemmer'] = df['token'].apply(lambda x: [stemmer.stem(word) for word in x])
# 2-4 정규화
# 영어인경우
import re
def normalize_text(text):
  # 소문자 변환
  text = text.lower()
  # 숫자제거  2024년 12월  25살 홍길동 .......
  # 숫자가 있었다는 사실만 알리기 위해서  0년 0월  0살 홍길동 .......
  text = re.sub(r'\d+', '0',text)  # \d+ 숫자가 연속적인 패턴
  # 특수문자 제거
  text = re.sub(f'[^\w\s]', '', text)  # \w --> [a-zA-Z0-9]   \s--> 스페이스,탭,줄바꿈
  return text
df['normalize'] = df['text'].apply(normalize_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# 피처 엔지니어링 tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['normalize'])
# 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['label'], test_size=0.2, random_state=42)
# 모델
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# 평가
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(f'report : {report}')

report :               precision    recall  f1-score   support

           0       0.89      0.88      0.89      2515
           1       0.88      0.90      0.89      2485

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000



Word2Vec 학습
```
소규모 데이터셋에서 품질이 낮음, 사전학습된 모델을 사용
```

In [ ]:
from gensim.models import Word2Vec
# 데이터 : 토큰화된 데이터
texts = [['영화','재미','최고'],['실망','별로']]
# word2vec 학습
model = Word2Vec(sentences=texts, vector_size=100, window=5,min_count=1)
print( model.wv.most_similar('영화', topn=2) )
# 벡터
model.wv['영화'][:10]

[('최고', 0.17018885910511017), ('실망', 0.004503022879362106)]


array([-0.00713902,  0.00124103, -0.00717672, -0.00224462,  0.0037193 ,
        0.00583312,  0.00119818,  0.00210273, -0.00411039,  0.00722533],
      dtype=float32)

CNN을 이용한 텍스트 분류

In [ ]:
# 토큰화된 데이터 필요
!wget http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
!wget http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
df = pd.read_csv('ratings_train.txt',sep='\t')
df.dropna(inplace=True)
df[:3]

--2025-04-22 02:14:18--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 3.5.186.208, 52.219.206.51, 3.5.184.115, ...
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|3.5.186.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt.1’

ratings_train.txt.1 100%[===================>]  13.95M  6.89MB/s    in 2.0s    

2025-04-22 02:14:21 (6.89 MB/s) - ‘ratings_train.txt.1’ saved [14628807/14628807]

--2025-04-22 02:14:21--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 3.5.184.20, 3.5.186.188, 3.5.186.61, ...
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|3.5.184.20|:80... connected.
HTTP request sent, aw

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


1. 전처리
2. 토큰화 후 단어 사전 구성
3. 시퀀스를 인덱스로 변환 + 패딩
4. 텐서로변환
5. cnn 학습(TextCnn)

In [ ]:
# 라이브러리
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from tqdm import tqdm

In [ ]:
# 전처리 함수
def clean_text(text):
  text = re.sub(r'[^ㄱ-ㅎ가-힣0-9\s]', "",text)
  return text.strip()

df['document']   = df['document'].apply(clean_text)

# 토큰화 및 단어사전 생성
tokenized =  df['document'].apply(lambda x : x.split())
keys = set([word for tokens in tokenized for word in tokens])
vocab = { key:idx for idx, key in enumerate(['<PAD>','<UNK>']  + list(keys)) }
vocab

# 시퀀스를 인덱스로 변환 - pad_squence 역활
max_len = 20
def encode(tokens):
  ids = [vocab.get(t,1) for t in tokens]
  if len(ids) < max_len:
    ids += [0]*(max_len - len(ids))
  else:
    ids = ids[:max_len]
  return ids
encoded_X = tokenized.apply(encode)
labels = df['label'].values

class TextDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.tensor(X.tolist(), dtype=torch.long)
    self.y = torch.tensor(y, dtype=torch.float32)
  def __len__(self):
    return len(self.X)
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

dataset = TextDataset(encoded_X.values, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# cnn모델  TextCNN 논문에서 발최한 내용
class TextCnn(nn.Module):
  def __init__(self, vocab_size, embed_size,num_filters,filter_sizes,output_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.convs = nn.ModuleList(  # 다양한 크기의 ngram을 적용해서 특징을 추출
        [ nn.Conv1d(embed_size, num_filter, kernel_size=fs) for fs in filter_sizes ]
      )
    self.fc = nn.Linear(num_filters*len(filter_sizes), output_size)  # flattern 역활
    self.relu = nn.ReLU()
  def forward(self, x)   :
    x = self.embedding(x).transpose(1,2)  # B,embed_size.seq_leg
    # dim = 2 seq_len 에 대해서 max pooing의 역활  각 필터에서 가장 강한 특징 하나만 추출
    x = [  torch.max(self.relu(conv(x)),dim=2)[0] for conv in self.convs  ]
    x = torch.cat(x, dim=1)  # batch,num_filters 형태를 이어붙인
    x = self.fc(x)
    return x

In [ ]:
vocab_size = len(vocab)
embed_size = 100
num_filter = 128
filter_sizes = [2,3,4]
output_size = 1
model = TextCnn(vocab_size, embed_size,num_filter,filter_sizes,output_size)
loss_fc = nn.BCEWithLogitsLoss() # sigmoid 내장
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for epoch in range(2):
  model.to(device)
  total_loss = 0
  for X,y in tqdm(dataloader):
    X = X.to(device)
    y = y.to(device)
    y_pred = model(X).squeeze()
    loss = loss_fc(y_pred, y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss.item()
  print(f'epoch : {epoch+1}, loss : {total_loss}')

100%|██████████| 4688/4688 [1:05:18<00:00,  1.20it/s]


epoch : 1, loss : 2535.570388227701


 43%|████▎     | 2020/4688 [31:04<41:02,  1.08it/s]


KeyboardInterrupt: 

전처리
```
한글 : 형태소분석, 불용어 제거, 정규화(특수문자제거), 어간/표제어 추출
영어 : 토큰화, 불용어 제거, 소문자 변환, 정규화, 어간/표제어 추출(특수문자제거)
공통 : 파이프라인 구축
```

In [ ]:
texts = ['''이틀 연속 무안타는 그의 사전에 없었다.
샌프란시스코 자이언츠의 이정후가 22일 밀워키 브루어스와의 MLB(미 프로야구) 홈 경기에서 2025시즌 두 번째 3루타를 쳤다.
3번 타자 겸 중견수로 선발 출전한 그는 3-2로 앞서던 7회 말 2사 1루에서 네 번째 타석에 섰다. 상대는 밀워키의 4번째 투수 제러드 케이닉이었다. 이때 밀워키의 투수 코치가 마운드에 올라갔다. 좌완 케이닉에게 이정후를 조심하라는 조언을 하기 위해서였다. 이정후는 전날까지 왼손 투수를 상대로 0.429라는 높은 타율을 기록 중이었다. 오른손 투수와 대결했을 때의 타율 0.283보다 훨씬 높다. 좌타자가 좌투수에 약하다는 상식과는 정반대다.
이정후는 케이닉이 1볼에서 던진 시속 150.5km짜리 싱커가 가운데로 낮게 들어오자 강하게 받아쳤다. 타구는 시속 164.5km로 외야 우중간을 갈랐다. 1루 주자 윌리 아다메스를 홈으로 불러들이는 적시타였다. 이정후는 3루까지 달렸다. 시즌 15번째 장타(홈런 3개·3루타 2개·2루타 10개).
이정후는 전날 LA 에인절스 원정에서 5타수 무안타에 그친데 이어 이날도 첫 세 타석까지 안타를 치지 못했다. 1회 2루수 땅볼, 3회 유격수 땅볼, 5회엔 2루 땅볼이었다.
이틀간 8타수 무안타는 지난 15일 필라델피아 필리스전(5타수 무안타)과 16일 필라델피아전의 첫 두 타석까지 당했던 7타수 무안타보다 더 긴 침묵이었다. 하지만 이번 시즌 무안타 경기가 4번에 불과했던 이정후는 2경기 연속 무안타에 대한 걱정을 시원한 장타로 날려버렸다.
시즌 22번째 출전이었던 22일을 4타수 1안타(1타점)로 마감한 이정후의 타율은 0.333에서 0.329(85타수 28안타·15타점· 20득점)로 약간 낮아졌다. 출루율(0.383)과 장타율(0.600)을 합친 OPS는 0.982에서 0.983으로 올라갔다.
샌프란시스코는 5대2로 이기며 내셔널리그 서부지구 3위(15승8패)를 유지했다.
'''
,
'''전 세계를 강타한 ‘지브리풍’ 그림 열풍이 인공지능(AI) 대중화의 길을 열었다는 평가와 함께 오락용 AI에 전력을 낭비한다는 비판의 목소리가 커지고 있습니다.
챗GPT는 개편 이후 첫 1주일간 7억 장의 이미지를 생성하는 데만 미국 6만7000가구가 하루에 쓰는 전력을 사용했습니다.
현지 테크업계에선 “뜻밖의 지브리 열풍이 범용인공지능(AGI) 시대 인류에 닥칠 문제점을 보여주는 계기가 됐다”는 평가입니다.
지난 2년간 AI 경쟁은 텍스트 데이터만을 기반으로 한 대규모언어모델(LLM) 개발에 초점이 맞춰졌습니다.
저작권 및 보안 이슈가 불거지면서 AI 훈련에 필요한 텍스트 데이터를 확보하는 데 한계에 직면했습니다.
빅테크들이 이미지, 영상, 음성 등 다양한 형태의 데이터 확보에 사활을 걸 수밖에 없는 이유입니다.
하지만 AI 창업자들이 수익 모델에 매달리면서 그들의 ‘거대한 포부’가 인류를 위기에 빠트릴 수 있다는 비관론이 퍼지고 있습니다.
미국 카네기멜런대와 허깅페이스의 연구에 따르면 AI로 이미지 한 장을 생성하는 데 평균 2.907와트시(Wh)의 전력이 소모됩니다.
이는 챗GPT에 단어 100개 미만의 짧은 텍스트 질문을 할 때 드는 전력 소모량(0.3Wh)의 약 열 배에 달합니다.
AI 개발 경쟁의 무게 중심이 텍스트에서 이미지와 영상으로 급격히 이동하는 현 상황은 전력난을 가속화할 수밖에 없습니다.
2015년 이세돌과 알파고 간 바둑 대전 당시에도 이 같은 논란이 불거졌습니다.
당시 알파고는 이세돌과 비교해 2만2000배가량의 전력을 소비했다는 분석이 나왔습니다.
현재 논란의 핵심은 고작 사람들의 SNS 프사를 위해 석유와 가스를 퍼올리고, 숲을 없애며, 수자원을 낭비하는 행위가 합리적이냐는 것입니다.
엘사 올리베티 MIT 교수는 “AI를 사용할 때 자신이 누르는 버튼 하나가 얼마나 많은 자원을 소모하는지 인식할 필요가 있다”고 지적했습니다.'''
,
         '''최근 직장인 온라인 커뮤니티 ‘블라인드’ 등에서는 5월 2일 임시공휴일로 지정되는 것이 맞느냐는 글이 올라오고 있다. 소셜미디어 엑스에서도 ‘임시공휴일’이 대한민국 트렌드 태그로 떠오르며 4000개 이상의 글이 올라오고 있다.
올해 5월 1일 근로자의 날은 목요일로 2일이 금요일이다. 3~4일은 주말이며, 5일은 부처님 오신 날이자 어린이날, 6일은 대체 공휴일이다. 이에 2일이 임시공휴일로 지정되면 총 엿새를 쉴 수 있다.
아직 정부에서는 임시공휴일 지정 여부에 대한 공식적인 결정을 내리지 않았다. 오히려 6월 3일 조기 대통령 선거일이 임시공휴일로 지정되면서, 연이은 공휴일 지정에 부담을 느끼고 있는 것으로 알려졌다.
하지만 올 초 정부가 설 연휴를 2주 앞두고 1월 27일 임시공휴일을 지정한 것을 두고, 이번에도 그럴 가능성이 있다는 누리꾼들의 의견이다.
임시공휴일에 대한 찬반도 있다. 직장인들은 5월 2일이 임시공휴일로 지정되면 엿새를 연차 없이 쉴 수 있어 환영하는 분위기다. 이와는 반대로 “너무 노는 것 아닌가, 그만 쉬자” “근로자의 날이나 제대로 쉬게 해줘라”는 의견도 보인다.
또한 연휴를 코앞에 앞두고 임시공휴일 지정 여부를 결정하는 것에 불만을 표하는 이들도 있다. 누리꾼들은 “당장 2주 후인데 ‘쉰다’ ‘안 쉰다’를 모르면 업무 일정에 차질이 생긴다” “휴일에 일해야 하는 곳은 급하게 사람을 구해야 한다”고 했다.
자영업자들도 5월 2일 임시공휴일을 반기지 않는다. 연휴가 길어지면 오히려 가게 영업에 도움이 되지 않기 때문이다. 앞서 구정 설날에도 정부는 내수 진작과 민생 회복의 취지로 임시공휴일을 지정했다. 하지만 기대와는 달리 해외로 출국하는 사람들이 6년 만에 최대 규모를 찍으며 그 효과를 보지 못해 비관론이 앞서고 있다.
임시공휴일은 대통령령 제24828호 ‘관공서의 공휴일에 관한 규정’에 따라 정부가 수시로 지정하는 공휴일이다. 특정한 목적에 따라 날짜를 지정한다. 근로기준법에 따라 2022년 1월부터 5인 이상 사업장은 임시 공휴일에 유급휴가를 줘야 하고, 이날 근무하면 주중 평일에 대체 휴가를 쓸 수 있다.
''']

In [ ]:
# 한글 전처리
'''
1. 데이터 : 인터넷 기사
2. 특수문자 제거(숫자,공백 제거)
3. 형태소 분석
4. 불용어 제거
5. 결과 반환
'''
def cleaned_text(text):
  text = re.sub(r'[^ㄱ-ㅎ가-힣0-9]\s', "",text)
  text = re.sub(r'\n|\r\n', "",text)
  return text

cleaned_texts = [cleaned_text(text) for text in texts]

In [ ]:
# 형태소 분석
from konlpy.tag import Okt
okt = Okt()
step1 = [okt.morphs(text) for text in cleaned_texts]
total=[]
for text in step1:
  temp = []
  for word in text:
    if len(word)>=2:
      temp.append(word)
  total.append(temp)
len(total)

3

In [ ]:
# 불용어 제거
import pandas as pd
url = 'https://gist.githubusercontent.com/spikeekips/40eea22ef4a89f629abd87eed535ac6a/raw/4f7a635040442a995568270ac8156448f2d1f0cb/stopwords-ko.txt'
stopword = pd.read_csv(url, header=None).values
stopword = [word for text in stopword for word in text]
cleaned_data = [word for word in total if word not in stopword]
cleaned_data = [' '.join(text) for text in cleaned_data]
cleaned_data

['이틀 연속 무안 타는 사전 없었다 샌프란시스코 자이언츠 이정후 22일 밀워키 브루어스 와의 MLB 프로야구 경기 에서 2025시 번째 루타 쳤다 타자 중견수 선발 출전 3-2 앞서던 7회 에서 번째 타석 섰다 상대 밀워키 번째 투수 제러드 케이 이었다 밀워키 투수 코치 마운드 올라갔다 좌완 케이 에게 이정후 조심 라는 조언 하기 해서였다 이정후 전날 까지 왼손 투수 상대로 0.429 라는 높은 타율 기록 이었다 오른손 투수 대결 했을 타율 0.283 보다 훨씬 높다 타자 투수 약하다는 상식 과는 정반대 이정후 케이 에서 던진 시속 150.5 km 짜리 싱커 가운데 낮게 들어오자 강하게 받아 쳤다 타구 시속 164.5 km 외야 우중 갈랐다 주자 윌리 메스 으로 불러 들이는 적시타 였다 이정후 까지 달렸다 시즌 15 번째 장타 홈런 루타 루타 10 이정후 전날 에인절스 원정 에서 타수 안타 그친데 이어 타석 까지 안타 했다 1회 루수 땅볼 3회 유격수 땅볼 5회 땅볼 이었다 이틀 타수 무안 타는 지난 15일 필라델피아 필리스 타수 안타 16일 필라델피아 전의 타석 까지 당했던 타수 안타 보다 침묵 이었다하지만 이번 시즌 안타 경기 불과했던 이정후 경기 연속 안타 대한 걱정 시원한 장타 날려 버렸다 시즌 22 번째 출전 이었던 22일 타수 안타 타점 마감 이정후 타율 0.333 에서 0.329 85 타수 28 안타 15 타점 20 득점 약간 낮아졌다 출루율 0.383 장타율 0.600 합친 OPS 0.982 에서 0.983 으로 올라갔다 샌프란시스코 이기 내셔널리그 서부 지구 15 유지 했다',
 '세계 강타 지브리 그림 열풍 인공 지능 AI 대중화 열었다는 평가 함께 오락 AI 전력 낭비한다는 비판 목소리 커지고 있습니다 GPT 개편 이후 일간 7억 장의 이미지 성하는 미국 6만 7000 가구 하루 쓰는 전력 사용 했습니다 현지 테크 업계 에선 뜻밖 지브리 열풍 범용 인공 지능 AGI 시대 인류 닥칠 문제점 보여주는 계기 됐다 평가 입니다지난 2년 경쟁 텍스

텍스트
```
분류(감정, 토픽)
텍스트요약, 키워드 추출, 개채명 인식
```

텍스트 요약
- 목표 : TF-IDF로문장 중요도를 계산해서 핵심문장 추출

In [18]:
from konlpy.tag import Okt
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
# 한글 전처리
import re
def preprocessing_kor(text):
  text = re.sub(r'[^ㄱ-ㅎ가-힣]\s', '', text)
  text = re.sub(r'\n|\r\n', '', text)
  okt = Okt()
  text = okt.morphs(text)
  return ' '.join(text)
def text_summary(text,num_seq=2):
  # 각 문장별 분리
  sentences = sent_tokenize(text)
  # 전처리
  preprocess_sentences = [preprocessing_kor(sentence) for sentence in sentences]
  # tf-idf
  tfidf = TfidfVectorizer()
  tfidf_matrix = tfidf.fit_transform(preprocess_sentences)
  # 문장점수 계산
  # sum 각 문장벡터의 tf-idf 합->높을수록 중요단어 많이포함
  sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
  # 상위문장 선택
  top_index = sentence_scores.argsort()[-num_seq:][::-1]  # 내림차순으로 정렬
  top_index = sorted(top_index)
  return ' '.join(sentences[i] for i in top_index)+'.'

In [79]:
text = '''
국내 연구진이 뇌가 시각 정보를 선택적으로 처리하는 방식으로 AI의 이미지 인식 능력을 높이는 기술을 개발했다.

기초과학연구원(IBS)은 이창준 인지 및 사회성 연구단장 연구팀은 송경우 연세대 교수팀과 공동으로 뇌의 시각피질이 시각 정보를 선별해 처리하는 방식을 응용해 AI의 이미지 인식 능력을 향상시키는 새로운 기술을 개발했다고 22일 밝혔다.

인간의 시각 시스템은 한 눈에 사물을 인식하고, 복잡한 환경에서도 중요한 정보를 빠르게 선별한다. 이에 비해 전통적인 AI 모델인 합성곱 신경망(CNN)은 작은 정사각형 필터로 이미지를 쪼개 분석하기 때문에 넓은 맥락을 파악하거나 거리가 떨어져 있는 정보 사이의 관계를 이해하는 데 한계가 있다. 이를 보완한 비전 트랜스포머는 막대한 연산량과 대규모 데이터 세트가 필요해 실용성에 제한이 있다.


연구팀은 인간 뇌의 시각 피질이 시각 정보를 선택적으로 처리하는 방식에 주목했다. 인간의 시각 피질은 모든 정보를 똑같이 처리하지 않고 눈에 띄는 특징이나 중요한 부분에만 집중해 선택적으로 반응한다. 이 과정에서 뉴런들은 넓은 범위에서 필요한 정보만 선택적으로 반응한다.

연구팀은 이런 인간 뇌의 뉴런 구조를 모사해 CNN 모델의 성능을 높일 수 있는 'Lp-컨볼루션' 기술을 제안했다. Lp-컨볼루션은 이미지에서 중요한 영역은 강조하고, 덜 중요한 영역은 덜 보이게 조정하는 가중치 필터 '마스크' 기술을 적용, AI가 이미지를 분석할 때 사람처럼 핵심적인 정보를 우선 파악할 수 있도록 설계됐다. 마스크는 학습 과정에서 스스로 형태를 조정해 다양한 환경에서도 중요한 특징에 집중할 수 있도록 지원한다.

연구팀은 Lp-컨볼루션을 다양한 CNN 모델에 적용해 성능을 평가한 결과, 기존 CNN 모델보다 이미지 분류 정확도가 눈에 띄게 향상됐음을 확인했다. 특히 한 번에 더 넓은 영역을 살펴볼 수 있도록 필터 크기를 넓혀도 성능 저하 없이 안정적으로 작동했고 정확도는 오히려 높아지는 결과를 얻었다.

일반적으로 분석 범위를 넓히면 계산량이 증가하고, 정확도가 떨어지는데, Lp-컨볼루션은 이런 한계를 극복한 것이다. 연구팀은 이어 Lp-컨볼루션이 실제 뇌의 정보 처리 방식과 얼마나 유사한지를 확인하는 실험에서도 기존 CNN 모델보다 뉴런 반응을 더 정밀하게 예측했으며, 예측 오차도 줄어든 것으로 나타났다.

이창준 IBS 단장은 "Lp-컨볼루션이 AI 성능 향상을 넘어 뇌가 정보를 어떻게 처리하는지를 모방학 이해하는 데 크게 기여할 수 있다"며 "AI와 뇌과학이 함께 발전할 수 있는 새로운 융합 모델의 좋은 사례가 될 것"이라고 말했다.
'''

In [80]:
text_summary(text)

'기초과학연구원(IBS)은 이창준 인지 및 사회성 연구단장 연구팀은 송경우 연세대 교수팀과 공동으로 뇌의 시각피질이 시각 정보를 선별해 처리하는 방식을 응용해 AI의 이미지 인식 능력을 향상시키는 새로운 기술을 개발했다고 22일 밝혔다. 이창준 IBS 단장은 "Lp-컨볼루션이 AI 성능 향상을 넘어 뇌가 정보를 어떻게 처리하는지를 모방학 이해하는 데 크게 기여할 수 있다"며 "AI와 뇌과학이 함께 발전할 수 있는 새로운 융합 모델의 좋은 사례가 될 것"이라고 말했다..'

키워드 추출

In [83]:
def extract_keyword(text, top_n=5):
  processed_text = preprocessing_kor(text)
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform([processed_text])
  feature_names = vectorizer.get_feature_names_out()
  tfidf_score = tfidf_matrix.toarray()[0]
  # tfidf 점수 기준으로 상위 top_n 개의 단어 인덱스를 내림차순
  sorted_index = tfidf_score.argsort()[-top_n:][::-1]
  return [(feature_names[i],tfidf_score[i])  for i in sorted_index]

In [84]:
keywords = extract_keyword(text, top_n=5)
for keyword, score in keywords:
  print(f'keyword : {keyword}, score : {score}')

keyword : 정보, score : 0.32091527190283997
keyword : 으로, score : 0.288823744712556
keyword : 하는, score : 0.224640690331988
keyword : 이미지, score : 0.192549163141704
keyword : 컨볼루션, score : 0.192549163141704


In [15]:
!unzip /content/drive/MyDrive/data/newsData.zip

Archive:  /content/drive/MyDrive/data/newsData.zip
replace newsData/0/0147NewsData.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: newsData/0/0147NewsData.txt  
  inflating: __MACOSX/newsData/0/._0147NewsData.txt  
  inflating: newsData/0/0025NewsData.txt  
  inflating: __MACOSX/newsData/0/._0025NewsData.txt  
  inflating: newsData/0/0137NewsData.txt  
  inflating: __MACOSX/newsData/0/._0137NewsData.txt  
  inflating: newsData/0/0055NewsData.txt  
  inflating: __MACOSX/newsData/0/._0055NewsData.txt  
  inflating: newsData/0/0194NewsData.txt  
  inflating: __MACOSX/newsData/0/._0194NewsData.txt  
  inflating: newsData/0/0169NewsData.txt  
  inflating: __MACOSX/newsData/0/._0169NewsData.txt  
  inflating: newsData/0/0086NewsData.txt  
  inflating: __MACOSX/newsData/0/._0086NewsData.txt  
  inflating: newsData/0/0119NewsData.txt  
  inflating: __MACOSX/newsData/0/._0119NewsData.txt  
  inflating: newsData/0/0105NewsData.txt  
  inflating: __MACOSX/newsData/0/._0105NewsData.txt  

In [16]:
# 데이터프레임 또는 넘파이나 리스트형태로 데이터와 라벨  X, y
from glob import glob
files = glob('/content/newsData/**/*.*',recursive=True)

texts = []
labels = []

for path in files:
  with open(path,'r') as f:
    label = int(path.split('/')[-2])
    text = f.read().strip()
    texts.append(text)
    labels.append(label)

# df = pd.DataFrame({'text':texts, 'label':labels})

In [19]:
# 전처리
from tqdm import tqdm
preprocessed_texts = []
for text in tqdm(texts):
  preprocessed_texts.append(preprocessing_kor(text))

100%|██████████| 1600/1600 [03:02<00:00,  8.74it/s]


In [20]:
# 데이터 분리(학습용과 검증용)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(preprocessed_texts, labels, test_size=0.2, random_state=42)
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [21]:
# 예측 및 평가
from sklearn.metrics import classification_report
y_pred = model.predict(X_test_tfidf)
report = classification_report(y_test, y_pred)
print(f'report : {report}')

report :               precision    recall  f1-score   support

           0       0.86      0.91      0.89        35
           1       0.77      0.86      0.81        35
           2       0.74      0.85      0.79        34
           3       0.94      0.69      0.80        48
           4       0.89      0.70      0.79        47
           5       0.87      0.98      0.92        41
           6       0.86      1.00      0.93        44
           7       1.00      1.00      1.00        36

    accuracy                           0.87       320
   macro avg       0.87      0.87      0.87       320
weighted avg       0.87      0.87      0.86       320



In [22]:
# 새로운 텍스트 예측
new_text = '''
최저임금위원회가 내년도 최저임금을 결정하기 위한 첫 번째 논의를 22일 시작했다. 특히 올해는 6월 3일 대통령 선거와 미국 트럼프 행정부의 관세 인상 압박이 맞물린 만큼, 최저임금에 대한 사회적 관심이 어느 때보다 커질 것으로 보인다.

최임위 논의의 최대 쟁점은 ①최저임금 인상률 ②최저임금 적용 대상 확대 ③최저임금 차등 적용 등이다. 이날 첫 회의부터 근로자(노동계) 측과 사용자(경영계) 측은 주요 쟁점마다 입장차를 드러내며 대립각을 세웠다.


우선 최저임금 인상률과 관련, 양측 모두 요구안을 확정하지 않은 상태다. 다만 근로자 측은 가파른 물가상승률 등을 고려해 올해 최저임금 협상 당시 제시했던 시간당 1만2,600원보다 높은 숫자를 요구할 전망이다. 서민음식으로 불린 순댓국밥 한 그릇 가격도 1만 원을 훌쩍 넘는 경우가 있는 만큼, 현재의 최저임금 1만30원으로는 일상적 생활이 불가능하다는 인식이다.

근로자위원인 이미선 민주노총 부위원장은 이날 회의에서 "노동자 실질임금이 삭감되어서는 안 된다. 치솟은 물가상승에 투잡, 스리잡으로 내몰리는 노동자들의 생존을 보장해야 한다"며 최저임금 인상의 당위성을 강조했다.

반면 사용자 측은 동결 카드를 꺼낼 것으로 보인다. 사용자위원인 이명로 중기중앙회 인력정책본부장은 "내수가 살아날 가능성이 희박한 상황에서 그나마 버텨주던 수출도 (미국의) 관세 인상으로 큰 감소가 예상된다"며 "최저임금을 지급해야 하는 중소기업, 영세사업주의 지불능력이 크게 약화된 상태를 고려해야 한다"고 강조했다.

최저임금 확대 적용도 뜨거운 감자다. 근로자 측은 지난해부터 배달기사와 같은 플랫폼 노동자와 특수고용직 노동자에게도 최저임금을 적용하자고 주장했다. 지난해 논의 때는 플랫폼·특고노동자에 대한 최저임금 심의를 최임위에서 할 수 있는지를 두고 노사 양측이 치열하게 대립했다.

하지만 이후 고용노동부가 최임위에서 플랫폼·특고 노동자에 대한 최저임금을 논의할 수 있다는 유권해석을 내리면서 논의의 통로가 열렸다. 근로자위원인 류기섭 한국노총 사무총장은 "최저임금 사각지대에 놓여 있는 플랫폼, 프리랜서, 특수고용 노동자들에게도 최저임금이 최소한의 보편적인 안전장치로 기능할 수 있도록 개선되어야 할 때"라고 강조했다.

사용자 측은 경영계 숙원 과제인 '최저임금 차등 적용' 카드를 꺼내 들었다. 최저임금법 제4조 1항은 최저임금을 사업의 종류별로 구분해 정할 수 있다고 규정했다. 지난해 사용자 측은 택시 운송업과 편의점, 음식점업에 대한 최저임금 차등 적용을 주장한 바 있다. 최저임금 차등 적용은 최저임금 제도가 처음 도입된 1988년 한 차례 시행된 바 있지만, 노동계의 강력한 반발에 가로막혀 현재는 사실상 사문화된 조항이다.


다만 올해는 국민의힘이 조기 대선 국면에서 업종별·기업규모별 최저임금 차등 적용 공약을 검토하며 의제를 부각시키고 있어 노사 양측의 치열한 논쟁이 벌어질 것으로 예측된다. 사용자위원인 류기정 한국경영자총협회(경총) 전무는 "최근 몇 년간 내수부진이 계속되며 최저임금을 지불해야 하는 중소기업과 영세 소상공인의 어려움이 매우 극심하게 힘든 상황으로 빠져들고 있다"며 "업종별 최저임금 구분적용도 보다 진전된 결과를 반드시 도출할 때"라고 말했다.

내년도 최저임금은 김문수 전 고용노동부 장관이 '최저임금 심의요청서'를 최임위에 제출한 날(3월 31일)로부터 90일(6월 28일) 이내에 결정해야 한다. 하지만 최저임금 논의 시한이 지켜진 경우는 9차례에 불과했고, 올해는 노사가 대립하는 쟁점도 많은 만큼 논의 과정은 더 길어질 전망이다. 최임위 2차 전원회의는 다음 달 27일 열린다.
'''

정치 경제 사회 생활/문화 세계 기술

In [23]:
new_text = preprocessing_kor(new_text)
tfidf_new_text = tfidf.transform([new_text])
predict = model.predict(tfidf_new_text)[0]
predict

1

In [24]:
np.where(np.array(labels) == 5)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [25]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6, 7])